# Notebook to extract LMC and SMC Cadence

In [294]:
# Check the version of MAF - the output should be version 2.4.2 or higher.
# 'unknown' means you're running from a git repo version,
# and have made changes to your local code

import lsst.sims.maf
lsst.sims.maf.__version__

'2.13.0.sims-59-g0a27faef'

In [295]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [296]:
# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

from lsst.sims.maf.db import ResultsDb

In [297]:
import sys
print(sys.path)

['/Users/dagoret/MacOSX/GitHub/LSST/MyCadence/ExtractLMC', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/obs_lsst/19.0.0-80-g0bd2bb3+1/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/ts_dateloc/master-g60a36c8d46+46/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/ts_observatory_model/master-g3d307a5f45+46/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_almanac/master-g021b69e146+1/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_downtimeModel/master-g55f72efa65+17/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_cloudModel/master-ge3724df529+19/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/s

### Specify my other python path (my installation)

In [298]:
otherpythonpaths=['/Users/dagoret/anaconda3/lib/python37.zip', '/Users/dagoret/anaconda3/lib/python3.7', '/Users/dagoret/anaconda3/lib/python3.7/lib-dynload', '', '/Users/dagoret/.local/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/dagoret/.ipython']

In [299]:
for thepath in otherpythonpaths:
    sys.path.append(thepath) # go to parent dir

In [300]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Astronomy

In [301]:
gindex=0

In [302]:
Galaxy_Name=["lmc","smc"]

In [303]:
Galaxy_Delta_RA=[10.75,5.33]
Galaxy_Delta_DEC=[9.17,3.083]

In [304]:
GName=Galaxy_Name[gindex]
galaxy_delta_ra=Galaxy_Delta_RA[gindex]
galaxy_delta_dec=Galaxy_Delta_DEC[gindex]

## astroquery

In [305]:
import astroquery
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

In [306]:
from astroquery.simbad import Simbad
result_table = Simbad.query_object(GName)
result_table.pprint(show_unit=True)

MAIN_ID      RA        DEC    ... COO_QUAL COO_WAVELENGTH     COO_BIBCODE    
          "h:m:s"    "d:m:s"  ...                                            
-------- ---------- --------- ... -------- -------------- -------------------
NAME LMC 05 23 34.6 -69 45 22 ...        D              O 2003A&A...412...45P


In [307]:
Angle(result_table["RA"][0]+" hour")

<Angle 5.39294444 hourangle>

In [308]:
Angle(result_table["DEC"][0]+" degrees")

<Angle -69.75611111 deg>

In [309]:
galaxy_coord=SkyCoord(ra=result_table["RA"][0]+"hour",dec=result_table["DEC"][0]+ "degrees")
galaxy_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (80.89416667, -69.75611111)>

In [310]:
galaxy_ra=galaxy_coord.ra.deg
galaxy_dec=galaxy_coord.dec.deg

In [311]:
print(galaxy_ra,galaxy_dec)

80.89416666666666 -69.75611111111111


# Config

## Database selection

In [312]:
dbindex=2

In [313]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']
colors = {'u': 'cyan', 'g': 'g', 'r': 'y','i': 'r', 'z': 'm', 'y': 'k'}
Nfilt=len(filters)

In [314]:
database=["baseline2018a.db","baseline_v1.5_10yrs.db","baseline_2snaps_v1.5_10yrs.db"]

In [315]:
dbfileName=database[dbindex]

In [316]:
dbtag=dbfileName.split(".")[0]

In [317]:
dbtag

'baseline_2snaps_v1'

# Prepare access to database

## SQL Selection

In [318]:
raCol = 'fieldRA'
decCol = 'fieldDec'

In [319]:
sqlWhere = \
    'fieldRA < %f and ' \
    'fieldRA > %f and ' \
    'fieldDEC < %f and ' \
    'fieldDEC > %f  ' \
     % (galaxy_ra+ galaxy_delta_ra/2., galaxy_ra- galaxy_delta_ra/2.,galaxy_dec+galaxy_delta_dec/2.,galaxy_dec-galaxy_delta_dec/2)

In [320]:
sqlWhere

'fieldRA < 86.269167 and fieldRA > 75.519167 and fieldDEC < -65.171111 and fieldDEC > -74.341111  '

## Metric

In [321]:
dbfields=['observationId','night','observationStartMJD','fiveSigmaDepth','filter','airmass','skyBrightness','seeingFwhmEff','fieldId','fieldRA','fieldDec']

In [322]:
metric=metrics.PassMetric(cols=dbfields)

##  Database to open

In [323]:
opsdb = db.OpsimDatabase('/Users/dagoret/DATA/Cadence/'+dbfileName)
outDir = 'output_galaxycadence'
resultsDb = db.ResultsDb(outDir=outDir)

## Slicer

In [324]:
slicer_uni=slicers.UniSlicer()
slicer_point = slicers.UserPointsSlicer(galaxy_ra, galaxy_dec, lonCol='fieldRA', latCol='fieldDec', latLonDeg='degrees')
nside = 512
slicer_heal = slicers.HealpixSlicer(nside=nside, lonCol=raCol, latCol=decCol)

Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


# Run

## Bundle

- Bundles dict for all filter

In [325]:
bundles = {}
bundles[GName] = metricBundles.MetricBundle(metric, slicer_uni, sqlWhere)
print(bundles[GName].dbCols)

{'skyBrightness', 'observationId', 'night', 'fieldRA', 'filter', 'airmass', 'observationStartMJD', 'seeingFwhmEff', 'fiveSigmaDepth', 'fieldId', 'fieldDec'}


In [326]:
bgroup = metricBundles.MetricBundleGroup(bundles, opsdb, outDir=outDir,resultsDb=resultsDb)

In [327]:
bgroup.runAll()
bgroup.plotAll(closefigs=False)

Querying database SummaryAllProps with constraint fieldRA < 86.269167 and fieldRA > 75.519167 and fieldDEC < -65.171111 and fieldDEC > -74.341111   for columns ['skyBrightness', 'observationId', 'night', 'fieldRA', 'filter', 'airmass', 'observationStartMJD', 'seeingFwhmEff', 'fiveSigmaDepth', 'fieldId', 'fieldDec']
Found 884 visits
Running:  ['lmc']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting figures with "fieldRA < 86.269167 and fieldRA > 75.519167 and fieldDEC < -65.171111 and fieldDEC > -74.341111  " constraint now.
Plotting complete.


/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_maf/2.13.0.sims-59-g0a27faef/python/lsst/sims/maf/db/resultsDb.py:279: UserWarning: Warning! Cannot save non-conforming summary statistic.
  warnings.warn('Warning! Cannot save non-conforming summary statistic.')


In [328]:
print(bundles[GName].dbCols)

{'skyBrightness', 'observationId', 'night', 'fieldRA', 'filter', 'airmass', 'observationStartMJD', 'seeingFwhmEff', 'fiveSigmaDepth', 'fieldId', 'fieldDec'}


In [329]:
print(len(bundles[GName].metricValues[0]))

884


# Output

In [330]:
pdfilename="cadence_"+GName+"_"+dbtag+".csv"

In [331]:
pdfilename

'cadence_lmc_baseline_2snaps_v1.csv'

In [332]:
df  = pd.DataFrame()

In [333]:
N=len(dbfields)

In [334]:
for index in np.arange(N):
    fieldname=dbfields[index]
    data=bundles[GName].metricValues[0][fieldname]
    df[fieldname]=data

In [335]:
df

,observationId,night,observationStartMJD,fiveSigmaDepth,filter,airmass,skyBrightness,seeingFwhmEff,fieldId,fieldRA,fieldDec
0,1923,3,59856.353356,24.032268,r,1.281545,21.065518,1.022068,-1,75.645541,-68.796842
1,1996,3,59856.387608,23.535707,i,1.242388,19.602172,0.770412,-1,77.676780,-66.549905
2,2774,4,59857.345243,23.968351,r,1.370114,21.009206,1.049511,-1,81.703694,-72.723577
3,5118,7,59860.324514,22.058213,y,1.331340,18.122261,0.832718,-1,76.870936,-70.849632
4,5151,7,59860.339312,21.959946,y,1.322429,18.133994,0.917603,-1,76.870936,-70.849632
...,...,...,...,...,...,...,...,...,...,...,...
879,1906366,3448,63270.075081,24.359166,r,1.370181,20.982480,0.724566,-1,84.816239,-73.092064
880,1906375,3448,63270.080472,23.714679,u,1.241531,22.633160,0.804692,-1,85.143745,-65.964410
881,1906408,3448,63270.097051,24.824610,g,1.255346,22.158383,0.783529,-1,85.143745,-65.964410
882,1912376,3457,63279.048885,23.208395,i,1.275802,20.131322,1.300346,-1,81.651530,-67.983175


In [336]:
df.to_csv(pdfilename)

In [337]:
!ls -l -t

total 976
-rw-r--r--  1 dagoret  staff  132955 Jan 21 22:09 cadence_lmc_baseline_2snaps_v1.csv
drwxr-xr-x  7 dagoret  staff     224 Jan 21 22:09 output_galaxycadence
-rw-r--r--  1 dagoret  staff   38837 Jan 21 22:09 ExtractLMCCadence.ipynb
-rw-r--r--  1 dagoret  staff  134755 Jan 21 22:09 cadence_lmc_baseline_v1.csv
-rw-r--r--  1 dagoret  staff  102924 Jan 21 22:08 cadence_lmc_baseline2018a.csv
-rw-r--r--  1 dagoret  staff   17009 Jan 21 22:05 cadence_smc_baseline_v1.csv
-rw-r--r--  1 dagoret  staff   20717 Jan 21 22:05 cadence_smc_baseline_2snaps_v1.csv
drwxr-xr-x  4 dagoret  staff     128 Jan 21 21:57 ExtractLMC
-rw-r--r--  1 dagoret  staff    6508 Jan 21 21:55 ExtractLMC.zip
